In [1]:
import numpy as np
import tensorflow as tf  #TF 1.1.0rc1
tf.logging.set_verbosity(tf.logging.ERROR)
import matplotlib.pyplot as plt
from tsc_model import Model,sample_batch

In [2]:
import collections

class Vocabulary(object):

  UNK_TOKEN = "<unk>"

  def __init__(self, tokens, size=None):
    self.unigram_counts = collections.Counter(tokens)
    # leave space for "<s>", "</s>", and "<unk>"
    top_counts = self.unigram_counts.most_common(None if size is None else (size - 1))
    vocab = ([self.UNK_TOKEN] +
             [w for w,c in top_counts])

    # Assign an id to each word, by frequency
    self.id_to_word = dict(enumerate(vocab))
    self.word_to_id = {v:k for k,v in self.id_to_word.iteritems()}
    self.size = len(self.id_to_word)
    if size is not None:
        assert(self.size <= size)

    # For convenience
    self.wordset = set(self.word_to_id.iterkeys())

    # Store special IDs
    self.UNK_ID = self.word_to_id[self.UNK_TOKEN]

  def words_to_ids(self, words):
    return [self.word_to_id.get(w, self.UNK_ID) for w in words]

  def ids_to_words(self, ids):
    return [self.id_to_word[i] for i in ids]

  def ordered_words(self):
    """Return a list of words, ordered by id."""
    return self.ids_to_words(range(self.size))


In [3]:
def slice_up_words(words, window_size=10, step_size=1):
    slices = []
    num_words = len(words)
    for index in range(0, num_words, step_size):
        slice = words[index:index+window_size]
        if len(slice) == window_size:
            slices.append(slice)
        else:
            break
    return slices
        

slice_up_words(["hello", "I", "am", "mr", ".", "anderson", "what", "is", "your", "name", "?"], 3, 3)

[['hello', 'I', 'am'], ['mr', '.', 'anderson'], ['what', 'is', 'your']]

In [4]:
from os import listdir
import re

def canonicalize_digits(word):
    if any([c.isalpha() for c in word]): return word
    word = re.sub("\d", "DG", word)
    if word.startswith("DG"):
        word = word.replace(",", "") # remove thousands separator
    return word

def canonicalize_word(word):
    word = word.lower()
    return canonicalize_digits(word) # try to canonicalize numbers

def canonicalize_words(words):
    current = []
    for word in words.split(" "):   
        if word[-1] in (".", ',', '?', ';', '!'):
            punk = word[-1]
            current.append(punk)
            word = word[0:-1]

        word = canonicalize_word(word)
        current.append(word)
    return current
      
def load_train_data(window_size, step_size):
    train_data_dir = 'train_data'
    y = []
    X = []
    all_tokens = []
    author_to_id = {}
    for author_id, author in enumerate(listdir(train_data_dir)):
        author_to_id[author] = author_id
        author_path = "%s/%s" % (train_data_dir, author)
        for file_name in listdir(author_path):
            full_path = "%s/%s" % (author_path, file_name)
            y.append(author_id)            
            with open(full_path, "r") as f:
                current = canonicalize_words(f.read())
                all_tokens += current
                X.append(current)
                
    vocab = Vocabulary(all_tokens)

    for i, x in enumerate(X):
        X[i] = vocab.words_to_ids(x)

    expanded_y = np.array([])
    expanded_X = None
    for i, x in enumerate(X):
        slices = slice_up_words(x, window_size, step_size)
        if expanded_X is None:
            expanded_X = np.array(slices)
        else:
            expanded_X = np.append(expanded_X, np.array(slices), axis=0)
        expanded_y = np.append(expanded_y, np.array([y[i]] * len(slices)))

    return vocab, expanded_X, expanded_y, author_to_id

In [5]:
num_words_per_x = 15
vocab, X_train, y_train, author_to_id = load_train_data(num_words_per_x, num_words_per_x)
N,sl = X_train.shape
num_classes = len(np.unique(y_train))
print vocab.size
print author_to_id

33323
{'thomas_jefferson': 0, 'john_adams': 1, 'alexander_hamilton': 3, 'george_washington': 6, 'james_madison': 2, 'james_monroe': 4, 'john_jay': 5}


In [7]:
def load_eval_data(window_size, step_size, vocab):
    eval_data_dir = "unknown_data"
    eval_X = {}
    eval_y = {}
    for file_name in listdir(eval_data_dir):
        full_path = "%s/%s" % (eval_data_dir, file_name)
        with open(full_path, "r") as f:
            current = vocab.words_to_ids(canonicalize_words(f.read()))

        slices = slice_up_words(current, window_size, step_size)
        expanded_X = np.array(slices)
        id = file_name.split("_")[2].split(".")[0]
        eval_X[id] = expanded_X
        # working with the assumption that James Madison wrote all the disputed papers
        eval_y[id] = np.array([author_to_id['james_madison']] * len(slices))

    return eval_X, eval_y

eval_X, eval_y = load_eval_data(num_words_per_x, num_words_per_x, vocab)
print eval_y

{'20': array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]), '58': array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]), '49': array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 

In [ ]:
#Set these directories
#direc = './data'
#summaries_dir = '.'

"""Load the data"""
#ratio = np.array([0.8,0.9]) #Ratios where to split the training and validation set
#X_train,X_val,X_test,y_train,y_val,y_test = load_data(direc,ratio,dataset='ChlorineConcentration')
#N,sl = X_train.shape
#num_classes = len(np.unique(y_train))

In [8]:
"""Hyperparamaters"""
num_epochs = 1
batch_size = 50
dropout = 0.8
config = {    'num_layers' :    3,               #number of layers of stacked RNN's
              'hidden_size' :   120,             #memory cells in a layer
              'max_grad_norm' : 5,             #maximum gradient norm during training
              'batch_size' :    batch_size,
              'learning_rate' : .005,
              'sl':             sl,
              'num_classes':    num_classes}

max_iterations = int(np.floor((num_epochs * N) / batch_size) + 1)


In [9]:
epochs = np.floor(batch_size*max_iterations / N)
print('Train %.0f samples in approximately %d epochs in %d iterations' %(N,epochs, max_iterations))

Train 64054 samples in approximately 1 epochs in 1282 iterations


In [12]:
#Instantiate a model
model = Model(config)

Finished computation graph


In [18]:
import time

"""Session time"""
sess = tf.Session() #Depending on your use, do not forget to close the session
#writer = tf.summary.FileWriter(summaries_dir, sess.graph)  #writer for Tensorboard
sess.run(model.init_op)

cost_train_ma = -np.log(1/float(num_classes)+1e-9)  #Moving average training cost
acc_train_ma = 0.0
try:
    start = time.clock()
    for i in range(max_iterations):
        X_batch, y_batch = sample_batch(X_train,y_train,batch_size)
        #Next line does the actual training
        cost_train, acc_train,_ = sess.run([model.cost,model.accuracy, model.train_op],feed_dict = {model.input: X_batch,model.labels: y_batch,model.keep_prob:dropout})
        if i != 0 and i%100 == 0:
            elapsed_time = time.clock() - start
            print "%d group trained in %d seconds" % (i, elapsed_time)
            start = time.clock()
            if i == 100:
                print "Predicted Full Train Time: %d minutes" % (max_iterations * elapsed_time / 60)
                print "Predicted Full Train Time: %f hours" % (max_iterations * elapsed_time / (60 * 60))

        cost_train_ma = cost_train_ma*0.99 + cost_train*0.01
        acc_train_ma = acc_train_ma*0.99 + acc_train*0.01


except KeyboardInterrupt:
  pass
  
epoch = float(i)*batch_size/N
#print('Trained %.1f epochs, accuracy is %5.3f and cost is %5.3f'%(epoch,acc_val,cost_val))
print('Trained %.1f epochs'%(epoch))

#now run in your terminal:
# $ tensorboard --logdir = <summaries_dir>
# Replace <summaries_dir> with your own dir


100 group trained in 13 seconds
Predicted Full Train Time: 288 minutes
Predicted Full Train Time: 4.807146 hours
200 group trained in 13 seconds
300 group trained in 13 seconds
400 group trained in 13 seconds
500 group trained in 13 seconds
600 group trained in 13 seconds
Trained 0.5 epochs


In [71]:
#Evaluate validation performance
# This is classifying batchs from disputed paper #20

total_acc = 0
for key, value in eval_X.iteritems():
    print "Evaluating federalist paper #%s" % key
    for i in range(1000):
        X_batch, y_batch = sample_batch(value,eval_y[key],batch_size)
        predictions, acc_val = sess.run([model.predictions, model.accuracy],feed_dict = {model.input: X_batch, model.labels: y_batch, model.keep_prob:1.0})
    #    print "Predictions:", predictions
    #    print "Suspected Actual: ", y_batch
    #    print "accuracy: %5.3f" % acc_val
        total_acc += acc_val
        if i != 0 and i % 100 == 0:
            print "Accuracy at iteration %d: %.2f" % (i, total_acc/i)
print total_acc


Evaluating federalist paper #20
Accuracy at iteration 100: 0.00
Accuracy at iteration 200: 0.00
Accuracy at iteration 300: 0.00
Accuracy at iteration 400: 0.00
Accuracy at iteration 500: 0.00
Accuracy at iteration 600: 0.00
Accuracy at iteration 700: 0.00
Accuracy at iteration 800: 0.00
Accuracy at iteration 900: 0.00
Evaluating federalist paper #58
Accuracy at iteration 100: 0.00
Accuracy at iteration 200: 0.00
Accuracy at iteration 300: 0.00
Accuracy at iteration 400: 0.00
Accuracy at iteration 500: 0.00
Accuracy at iteration 600: 0.00
Accuracy at iteration 700: 0.00
Accuracy at iteration 800: 0.00
Accuracy at iteration 900: 0.00
Evaluating federalist paper #49
Accuracy at iteration 100: 0.00
Accuracy at iteration 200: 0.00
Accuracy at iteration 300: 0.00
Accuracy at iteration 400: 0.00
Accuracy at iteration 500: 0.00
Accuracy at iteration 600: 0.00
Accuracy at iteration 700: 0.00
Accuracy at iteration 800: 0.00
Accuracy at iteration 900: 0.00
Evaluating federalist paper #55
Accuracy